# Idea to improve motion network accuracy

Essentially I've done two things I've used a simple CIFAR style network (i.e. small convolution network of the internet that scores over 90% on CIFAR) which gave me 25% accuracy. Then I attempted using VGG16 which is quite a powerful network but I fudged the input layer weights using a trick call network modality. 
In theory this mean fine scale structures are potentially going into further on layers looking like garbage. I still got some decent results. But the training was quite slow. 

I do feel perhaps I'm not being patient with the training. 10 classes is one thing (CIFAR), a 101 is another. And then 1000 classes is definitely something else. Learning more fine grained differences may simply take quite a long time of learnning. I'm considering a much longer amount of training to see how it evolves. 

In [1]:
import os
import sys


up1 = os.path.abspath('../utils/') 
up2 = os.path.abspath('../models/') 
sys.path.insert(0, up1)
sys.path.insert(0, up2)

In [2]:
from optical_flow_data_gen import DataGenerator
from ucf101_data_utils import get_test_data_opt_flow, get_train_data_opt_flow
from motion_network import getKerasCifarMotionModel2, getKerasCifarMotionModelOnly
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from keras.optimizers import SGD
import cv2 

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
motion_model=getKerasCifarMotionModel2((224,224,2), 101)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      608       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
__________

In [8]:
cifar_model = getKerasCifarMotionModelOnly((224,224,2), 101)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 32)      608       
_________________________________________________________________
activation_15 (Activation)   (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 222, 222, 32)      9248      
_________________________________________________________________
activation_16 (Activation)   (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 111, 111, 64)      18496     
__________

Its interesting between both models there's a significant drop in parameters mostly because I the use of further downsampling with convolution nets and pooling

In [4]:
params_train = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 32,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
           'validation' : False}

params_valid = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 32,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
           'validation' : True}

In [5]:
id_labels_train = get_train_data_opt_flow('../data/ucf101_splits/trainlist01.txt')
labels = id_labels_train[1]
id_test = get_test_data_opt_flow('../data/ucf101_splits/testlist01.txt', \
                       '../data/ucf101_splits/classInd.txt')

training_generator = DataGenerator(*id_labels_train, **params_train)


validation_generator = DataGenerator(id_test[0], id_test[1], **params_valid)

In [6]:
mypotim = SGD(lr=1e-3, momentum=0.9)
motion_model.compile(loss='categorical_crossentropy',
              optimizer=mypotim,
              metrics=['accuracy'])

In [7]:
mod1 = motion_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=4, epochs=1,
                    verbose=1)

Epoch 1/1
298/298 [==============================] - 161s 540ms/step - loss: 5.1435 - acc: 0.0122 - val_loss: 6.8398 - val_acc: 0.0090


In [8]:
mod1 = motion_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=4, epochs=5,
                    verbose=1)

Epoch 1/5
298/298 [==============================] - 137s 458ms/step - loss: 4.6601 - acc: 0.0124 - val_loss: 7.6602 - val_acc: 0.0093
Epoch 2/5
298/298 [==============================] - 138s 462ms/step - loss: 4.6552 - acc: 0.0130 - val_loss: 6.7622 - val_acc: 0.0124
Epoch 3/5
 61/298 [=====>........................] - ETA: 1:32 - loss: 4.6570 - acc: 0.0118

Process ForkPoolWorker-36:
Process ForkPoolWorker-34:
Process ForkPoolWorker-35:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*sel

KeyboardInterrupt: 